# Segmentação
Referências: <br>
https://scikit-image.org/docs/dev/auto_examples/segmentation/plot_segmentations.html <br>
https://scikit-image.org/docs/dev/api/skimage.segmentation.html
***

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
from skimage.segmentation import felzenszwalb,slic,quickshift,watershed
from skimage.color import rgb2gray
from skimage.filters import sobel

## Funcoes de imshow

In [ ]:
#minha funcao para mostrar as imagens
def imshow(im,big=False):
    if big:
        plt.figure(figsize=(20,20))
    else:
        plt.figure()
    if len(im.shape)<3:
        plt.imshow(im, cmap='gray')#As cores ficam zoadas no Jupyter se deixar sem esse parâmetro
    else:
        plt.imshow(im[:,:,::-1]) 
    plt.axis('off')
    plt.show()

In [ ]:
#pinta uma matriz de labels com cores diferentes
def pinta_blobs(labels,angle_delta=20):
    #cria imagens
    im_hsv = np.ones((labels.shape[0],labels.shape[1],3)).astype('uint8')*255
    
    #printa usando a mascara
    im_hsv[:,:,0] = (labels*angle_delta)%180

    return cv2.cvtColor(im_hsv, cv2.COLOR_HSV2BGR)

#pinta uma matriz de labels com a media de cores de im 
def pinta_blobs2(labels,im):
    n_labels = int(labels.max()+1)
    im_out = np.zeros(im.shape)
    
    #divide por canal
    b,g,r    = im[:,:,0],im[:,:,1],im[:,:,2]
    bo,go,ro = im_out[:,:,0],im_out[:,:,1],im_out[:,:,2]
    
    for label in range(n_labels):
        mask = labels==label
        bo[mask] += np.median(b[mask])#b[mask].mean()
        go[mask] += np.median(g[mask])#g[mask].mean()
        ro[mask] += np.median(r[mask])#r[mask].mean()
    return im_out.astype('uint8')

## Abre imagem

In [ ]:
im = cv2.imread('imgs/1.jpg')
imshow(im)

***
# 1. Minha Segmentacao
### 1.1 floodfill com distância de cores

In [ ]:
#distancia entre a cor de dois pixels em RGB
def dist_cor(pixel1,pixel2):
    #return np.sqrt(((pixel1-pixel2)**2).sum())
    return ((pixel1-pixel2)**2).sum()

In [ ]:
#floodfill com pilha
def floodfill(matrix,labels,x0,y0,label,alpha=50):
    size = matrix.shape
    
    pilha = list()
    pilha.append([x0,y0,None])
    
    while(pilha):
        
        x,y,pixel_ant = pilha.pop()
        labels[y,x] = label
        
        # Se as cores forem parecidas
        if pixel_ant is None or dist_cor(matrix[y,x],pixel_ant) <= alpha:
            #recursivo para os vizinhos
            if x > 0 and labels[y,x-1]<0:
                pilha.append([x-1,y,matrix[y,x]])
            if x < size[1] - 1 and labels[y,x+1]<0:
                pilha.append([x+1,y,matrix[y,x]])
            if y > 0 and labels[y-1,x]<0:
                pilha.append([x,y-1,matrix[y,x]])
            if y < size[0] - 1 and labels[y+1,x]<0:
                pilha.append([x,y+1,matrix[y,x]])

In [ ]:
def meu_segmenta(im):
    labels = np.ones((im.shape[0],im.shape[1])).astype(int)*-1
    label = 0
    for y in range(im.shape[0]):
        for x in range(im.shape[1]):
            if labels[y,x]==-1:
                floodfill(im,labels,x,y,label)
                label+=1
    print(label)
    return labels

In [ ]:
%%time
labels = meu_segmenta(im)

In [ ]:
imshow(pinta_blobs(labels))
imshow(pinta_blobs2(labels,im))

### 1.2 

In [ ]:
#matriz de adjacência dos blobs
def matriz_adj(labels,viz8=False):
    #cria matriz
    size = int(labels.max()+1)
    adj = np.identity(size+1).astype(int)
    
    #labels com borda (para evitar de olhar limites nas vizinhanças)
    labels2 = np.ones(np.array(labels.shape)+2).astype(int)*size
    labels2[1:-1,1:-1] = labels
    
    for y in range(1,labels.shape[0]+1):
        for x in range(1,labels.shape[1]+1):
            adj[labels2[y,x],labels2[y,x-1]]+=1
            adj[labels2[y,x],labels2[y,x+1]]+=1
            adj[labels2[y,x],labels2[y-1,x]]+=1
            adj[labels2[y,x],labels2[y+1,x]]+=1
            if viz8:
                adj[labels2[y,x],labels2[y-1,x-1]]+=1
                adj[labels2[y,x],labels2[y-1,x+1]]+=1
                adj[labels2[y,x],labels2[y+1,x-1]]+=1
                adj[labels2[y,x],labels2[y+1,x+1]]+=1
                
    # retira vertices das bordas
    adj = adj[:-1,:-1]
    return adj
#matriz_adj(labels)

In [ ]:
#cor mediana de cada blob
def mediana_blobs(labels,im):
    n_labels = int(labels.max()+1)
    
    cores = np.zeros((n_labels,3)).astype(int)
    
    #divide os canais
    b,g,r    = im[:,:,0],im[:,:,1],im[:,:,2]
    
    for l in range(n_labels):
        mask = labels==l
        cores[l,0] = np.median(b[mask])#b[mask].mean()
        cores[l,1] = np.median(g[mask])#g[mask].mean()
        cores[l,2] = np.median(r[mask])#r[mask].mean()
    return cores
#mediana_blobs(labels,im)

In [ ]:
# dado dois blobs de label l1 e l2
# junta os dois e arruma a matriz de adjacência
def junta_blobs(l1,l2,labels,adj):
    labels = labels.copy()
    
    #arruma matriz labels
    labels[labels==l2] = l1
    labels[labels>l2] -= 1
    
    adj[l1,:] += adj[l2,:]
    adj[:,l1] += adj[:,l2]
    adj[l1,l1] -= 1
    
    adj=np.delete(adj, l2, 0)
    adj=np.delete(adj, l2, 1)
    
    return labels,adj

In [ ]:
def agrupa(im,labels,dist_cor_min=30,conexos_min=1):
    labels = labels.copy()
    
    adj = matriz_adj(labels)
    cores = mediana_blobs(labels,im)
    n_labels = cores.shape[0]
    n_labels_ant = n_labels+1
    
    #enquando conseguir diminuir o numero de labels
    while(n_labels_ant>n_labels):
        n_labels_ant = n_labels
        
        #para cada par de labels
        l1=0
        while(l1<n_labels):
            print(l1,'/',n_labels)
            l2 = l1+1
            while(l2<n_labels):
                if dist_cor(cores[l1],cores[l2])<=dist_cor_min and adj[l1,l2]>=conexos_min:
                    labels,adj = junta_blobs(l1,l2,labels,adj)
                    n_labels-=1
                else:
                    l2+=1
            l1+=1
        
        #novo vetor de cores
        cores = mediana_blobs(labels,im)
        n_labels = cores.shape[0]
        
        print(n_labels_ant,'para',n_labels,'blobs')
    return labels

In [ ]:
#labels = agrupa(im,labels)

***
# 2. felzenszwalb

In [ ]:
%%time
#segments_fz = felzenszwalb(im, scale=100, sigma=0.5, min_size=50)
segments_fz = felzenszwalb(im, scale=100, sigma=0.5, min_size=3)

In [ ]:
imshow(pinta_blobs(segments_fz))
imshow(pinta_blobs2(segments_fz,im))

***
# 3. Slic

In [ ]:
%%time
#segments_slic = slic(im, n_segments=250, compactness=10, sigma=1)
segments_slic = slic(im, n_segments=250, compactness=10, sigma=1)

In [ ]:
imshow(pinta_blobs(segments_slic))
imshow(pinta_blobs2(segments_slic,im))

***
# 4. quickshift

In [ ]:
%%time
#segments_quick = quickshift(img, kernel_size=3, max_dist=6, ratio=0.5)
segments_quick = quickshift(im, kernel_size=3, max_dist=6, ratio=0.5)

In [ ]:
imshow(pinta_blobs(segments_quick))
imshow(pinta_blobs2(segments_quick,im))

***
# 5. Watershed

In [ ]:
%%time
gradient = sobel(rgb2gray(im))
#segments_watershed = watershed(gradient, markers=250, compactness=0.001)
segments_watershed = watershed(gradient, markers=2000, compactness=0)

In [ ]:
imshow(pinta_blobs(segments_watershed))
imshow(pinta_blobs2(segments_watershed,im))